In [1]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 1.2 MB 26.0 MB/s 
     |████████████████████████████████| 5.8 MB 63.6 MB/s 
     |████████████████████████████████| 182 kB 75.6 MB/s 
     |████████████████████████████████| 7.6 MB 61.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [3]:
spacy.__version__

'3.4.4'

In [4]:
!nvidia-smi

Sat Dec 24 03:19:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |    336MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git


Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 16), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (82/82), done.


In [7]:
cv_data = json.load(open('/content/drive/MyDrive/resume/data/dataset.json','r'))

In [9]:
len(cv_data)

1014

In [ ]:
# cv_data


In [10]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
# cv_data[0]

In [42]:
import re
def cleanResume(x):
    # x = re.sub('http\S+\s', ' ', x)  # remove URLs
    x = re.sub('RT|cc', ' ', x)  # remove RT and cc
    x = re.sub('#\S+', '', x)  # remove hashtags
    x = re.sub('@\S+', '  ',x)  # remove mentions
    x = re.sub('[%s]' % re.escape("""!"#$%&'()+,:;<=>?@[]^`{|}~"""), ' ', x)
    # x = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[]^`{|}~"""), ' ', x)  # remove punctuations
    x = re.sub(r'[^\x00-\x7f]',r' ', x) 
    x = re.sub('\s+', ' ', x)  # remove extra whitespace
    x = x.lower()
    return x

In [67]:
def get_spacy_doc(file, data):
  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    text = cleanResume(text)
    print(text)
    # print(len(text))
    doc = nlp.make_doc(text)
    # print(type(doc))
    # break
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
        
      entity_indices = entity_indices + list(range(start,end))

      try:
         span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
          continue

      if span is None:
          err_data = str([start,end]) + "    " + str(text) + "\n"
          file.write(err_data)

      else:
          ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass
        
  return db

      

In [68]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

In [50]:
len(train), len(test)

(811, 203)

In [57]:
print(doc)

Contact www.linkedin.com/in/yubraj- upadhaya (LinkedIn) Top Skills DevOps Data Structures Algorithms Yubraj Upadhaya Backend Python Developer | Devops Learning Nepal Summary Data Structures and Algorithms... Python... Coding problem solving... Experience Stealth Startup 1 year 8 months Delivery Engineer Lead November 2021 - Present (1 year 2 months) Back End Developer May 2021 - October 2021 (6 months) Education Texas International College Bachelor studying, Java · (2017 - 2021) Tribhuvan University Bachelor's degree, BSC CSIT · (2017 - 2021) Page 1 of 1


In [69]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

  2%|▏         | 17/811 [00:00<00:04, 162.81it/s]

 contact www.linkedin.com/in/pawangaire linkedin top skills sql data analysis data entry certifications ask questions to make data-driven decisions introduction to data analysis using microsoft excel introduction to data analysis using microsoft excel ask questions to make data-driven decisions microsoft excel power query pawan gaire data analyst data entry data visualizations nawalparasi district nepal summary hi there it s me pawan gaire from bardaghat. i am currently learning about data analysis from coursera. i am so excited to understand what data is showing us. we can understand a lot more things about the company or organization just by analyzing it. i am so curious to unwrap the mysterious gift from the data. i am also looking forward to working as a junior data analyst. experience freelancer.com data analyst january 2020 - present 2 years 8 months education daunne devi high school 2 management march 2019 page 1 of 1
 contact www.linkedin.com/in/nitesh- ranjitkar-857293143 link

  7%|▋         | 55/811 [00:00<00:04, 159.73it/s]

 contact www.linkedin.com/in/sid-palas linkedin sidpalas.com personal creationautomation.com blog top skills matlab simulations solidworks certifications probabilistic graphical models 1 representation with honors probabilistic graphical models 3 learning probabilistic graphical models 2 inference honors-awards nsf graduate research fellowship mit school of engineering wheless fellowship tau pi sigma scholarship tedxlockheedmartin team special recognition award publications out of bounds additive manufacturing sid palas engineer devops directive llc san francisco bay area summary software engineer with a mechanical background experience devops directive llc software engineer february 2020 - present 2 years 7 months san francisco california united states i help companies and individuals solve their devops and cloud infrastructure challenges. i also write articles and create videos about those topics. i m experimenting with many different formats links below - prefer written content http

 11%|█         | 88/811 [00:00<00:05, 143.28it/s]

...

contact www.linkedin.com/in/bhupeshdubey-249781168 linkedin top skills web3.js solidity smart contracts certifications problem solving java se 7 programmer bhupesh dubey fullstack blockchain developer ethereum binance polygon solidity smart contracts hardhat truffle ether.js web3.js indore summary a software engineer with overall 2.5 years of industry experience. worked on different technologies like blockchain ethereum polygon smart contracts solidity web3.js ethers.js java node.js angular spring and many more being managing different stages of software development lifecycle. no matter what technology is used what i prioritize is the standard optimization and one of the most important client satisfaction reliability experience rapid innovation fullstack blockchain developer january 2022 - present 7 months india apptmyz fullstack developer march 2021 - october 2021 8 months hyderabad zensar technologies 6 months jr. software engineer november 2020 - december 2020 2 months hyderabad tel

 94%|█████████▍| 191/203 [00:02<00:00, 117.09it/s]

 contact www.linkedin.com/in/nabinchhetri linkedin nabinchhetri.com personal top skills computer science python programming language databases certifications introduction to cybersecurity test-driven development in django networking academy learn-a-thon 2020 become a django developer nabin chhetri python / django developer nepal page 1 of 1
 contact www.linkedin.com/in/sagar- neupane-000935222 linkedin top skills engineering teamwork communication certifications informatica power center microsoft azure fundamental sagar neupane data analyst lalitpur district nepal summary i am currently working as a data analyst on mindtree pvt ltd. i have a good experience on sql informatica spark databricks. i have been involved in creation of data flows testing and maintenance. experience mindtree data analyst september 2020 - present 2 years education chandigarh university bachelor of engineering - be computer science august 2016 - july 2020 page 1 of 1
 contact www.linkedin.com/in/mrsonam linkedin

100%|██████████| 203/203 [00:02<00:00, 85.83it/s] 



 contact www.linkedin.com/in/rbnp98 linkedin top skills machine learning artificial intelligence ai database design certifications machine learning engineering for production mlops specialization deeplearning.ai tensorflow developer specialization neural networks and deep learning tensorflow developer certificate sequence models rojen b.n pradhan machine learning engineer at fusemachines nepal experience fusemachines 2 years 6 months machine learning engineer february 2021 - present 1 year 6 months kathmandu b gmat nepal associate machine learning engineer may 2020 - february 2021 10 months kathmandu b gmat nepal associate machine learning engineer trainee february 2020 - april 2020 3 months kathmandu b gmat nepal leapfrog technology inc. software engineering intern november 2019 - december 2019 2 months kathmandu nepal xavier academy students council member 2014 - 2016 2 years education tribhuvan university- institute of science and technology bachelors of science in computer science

In [56]:
len(db.tokens)


203

In [19]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-24 03:22:04,176] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-24 03:22:04,186] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-12-24 03:22:04,190] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-24 03:22:04,191] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 501M/501M [00:05<00:00, 99.9MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFo

In [20]:
nlp = spacy.load('/content/output/model-best')

In [21]:
doc = nlp('My name is Roman Shilpakar. I worked at Google. I have 5 years of experience.')
for ent in doc.ents:
  print(ent.text, "  ->>>>  ", ent.label_)

In [22]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.0 MB 13.0 MB/s 


In [32]:
import sys, fitz

fname = '/content/drive/MyDrive/resume/data/test.pdf'
doc = fitz.open(fname)

In [ ]:
# doc = [page.get_text() for page in doc]

In [33]:
text = " "
for page in doc:
  text = text + str(page.get_text())

In [34]:
text = text.strip()

In [35]:
text = ' '.join(text.split())

In [36]:
text

"Contact www.linkedin.com/in/yubraj- upadhaya (LinkedIn) Top Skills DevOps Data Structures Algorithms Yubraj Upadhaya Backend Python Developer | Devops Learning Nepal Summary Data Structures and Algorithms... Python... Coding problem solving... Experience Stealth Startup 1 year 8 months Delivery Engineer Lead November 2021 - Present (1 year 2 months) Back End Developer May 2021 - October 2021 (6 months) Education Texas International College Bachelor studying, Java · (2017 - 2021) Tribhuvan University Bachelor's degree, BSC CSIT · (2017 - 2021) Page 1 of 1"

In [37]:
doc = nlp(text)
for ent in doc.ents:
  print(ent.text, "  ->>>>  ", ent.label_)

www.linkedin.com/in/yubraj- upadhaya   ->>>>   LINKEDIN LINK
DevOps Data Structures Algorithms   ->>>>   SKILLS
Yubraj Upadhaya   ->>>>   NAME
Python Developer   ->>>>   SKILLS
Summary Data Structures and Algorithms   ->>>>   SKILLS
Stealth Startup   ->>>>   COMPANIES WORKED AT
Delivery Engineer Lead   ->>>>   WORKED AS
1 year 2 months   ->>>>   YEARS OF EXPERIENCE
Back End Developer   ->>>>   WORKED AS
6 months   ->>>>   YEARS OF EXPERIENCE
Tribhuvan University   ->>>>   UNIVERSITY
